CAI DAT CAC THU VIEN

In [81]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


CAI DAT CAC THU VIEN CAN THIET

In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [83]:


data = pd.read_csv("strokeDataset.csv")
data = data[data['gender'] != 'Other']
mean_bmi = data['bmi'].mean()
data['bmi'] = data['bmi'].fillna(mean_bmi)

XỬ LÝ DỮ LIỆU

In [84]:
print(data['id'].isnull())

0       False
1       False
2       False
3       False
4       False
        ...  
5105    False
5106    False
5107    False
5108    False
5109    False
Name: id, Length: 5109, dtype: bool


In [85]:


target = "stroke"
x = data.drop(target, axis = 1)
x = x.drop("id", axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [86]:
gender_values = x_train["gender"].unique()
everMarried_values = x_train["ever_married"].unique()
workType_values = x_train["work_type"].unique()
ResidenceType_values = x_train["Residence_type"].unique()
smokingStatus_values = x_train["smoking_status"].unique()

In [87]:
num_transformer = Pipeline(steps=[
    ("impute", SimpleImputer(missing_values=-1, strategy="median")),
    ("scaler", StandardScaler())
])

In [88]:
nom_transformer = Pipeline(steps=[
    ("impute", SimpleImputer(missing_values=-1, strategy="most_frequent")),
    ("scaler", OneHotEncoder())
])

In [89]:
preprocessor = ColumnTransformer(transformers=[
    ("num_feature", num_transformer, ["age", "avg_glucose_level", "bmi"]),
    ("nom_feature", nom_transformer, ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]),
])

In [90]:
reg = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor())
])

In [92]:
params = {
    "preprocessor__num_feature__impute__strategy": ["median", "mean"],
    "model__n_estimators": [100,200,300],
    "model__criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"]
}
grid_search = GridSearchCV(estimator=reg, param_grid=params, cv=2, scoring="r2", verbose=2, n_jobs=3)
grid_search.fit(x_train, y_train)
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)
y_predict =grid_search.predict(x_test)
print("MAE: {}".format(mean_absolute_error(y_test, y_predict)))
print("MSE: {}".format(mean_squared_error(y_test, y_predict)))
print("R2: {}".format(r2_score(y_test, y_predict)))

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[CV] END model__criterion=squared_error, model__n_estimators=100, preprocessor__num_feature__impute__strategy=mean; total time=   1.5s
[CV] END model__criterion=squared_error, model__n_estimators=100, preprocessor__num_feature__impute__strategy=median; total time=   1.5s
[CV] END model__criterion=squared_error, model__n_estimators=100, preprocessor__num_feature__impute__strategy=median; total time=   1.8s
[CV] END model__criterion=squared_error, model__n_estimators=100, preprocessor__num_feature__impute__strategy=mean; total time=   1.9s
[CV] END model__criterion=squared_error, model__n_estimators=200, preprocessor__num_feature__impute__strategy=median; total time=   3.2s
[CV] END model__criterion=squared_error, model__n_estimators=200, preprocessor__num_feature__impute__strategy=median; total time=   3.5s
[CV] END model__criterion=squared_error, model__n_estimators=200, preprocessor__num_feature__impute__strategy=mean; total time=   2.9s
[CV] END model__criterion=squared_error, model_

KeyboardInterrupt: 